In [19]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Carica i dati
file_path = '../newData/Shifted_playoff.csv'
df = pd.read_csv(file_path)

# Dividi i dati per l'allenamento (fino all'anno 8) e per il test (anno 9)
train_data = df[df['year'] < 9]  # Anni fino al 8 per allenare il modello
test_data = df[df['year'] == 9]  # Anno 9 per testare il modello
predict_data = df[df['year'] == 10]  # Anno 10 per fare la predizione

# Definisci le feature e la variabile target per il training e il test
X_train = train_data.drop(columns=['PlayOffNextYear', 'tmID', 'year'])
y_train = train_data['PlayOffNextYear']
X_test = test_data.drop(columns=['PlayOffNextYear', 'tmID', 'year'])
y_test = test_data['PlayOffNextYear']

# Crea e addestra il modello Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Fai previsioni sui dati di test
y_pred = model.predict(X_test)

# Calcola le metriche di valutazione
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Ora fai previsioni sull'anno 10
X_predict = predict_data.drop(columns=['PlayOffNextYear', 'tmID', 'year'])
predict_data['PlayoffProbability'] = model.predict_proba(X_predict)[:, 1]

# Normalizza le probabilità per ottenere una somma di 8
predict_data['AdjustedPlayoffProbability'] = 8 * predict_data['PlayoffProbability'] / predict_data['PlayoffProbability'].sum()

# Aggiungi la colonna per la previsione finale di qualificazione
predict_data['PredictedPlayoffNextYear'] = (predict_data['AdjustedPlayoffProbability'] >= 0.5).astype(int)

# Seleziona le prime 4 squadre per conference (East e West) in base alla probabilità aggiustata
east_teams = predict_data[predict_data['confID'] == 1].nlargest(4, 'AdjustedPlayoffProbability')
west_teams = predict_data[predict_data['confID'] == 0].nlargest(4, 'AdjustedPlayoffProbability')

# Unisci le squadre selezionate per i playoff
playoff_teams = pd.concat([east_teams, west_teams])

# Visualizza le squadre previste per i playoff nell'anno 10
print("Squadre previste per i playoff nell'anno 10:")
print(playoff_teams[['tmID', 'confID', 'PlayoffProbability', 'AdjustedPlayoffProbability', 'PredictedPlayoffNextYear']])

# Calcola l'errore tra la previsione normalizzata e la colonna reale 'PlayOffNextYear'
predict_data['PlayOffNextYear'] = predict_data['PlayOffNextYear'].fillna(0)  # Riempie eventuali NaN con 0 per il calcolo
predict_data['Error'] = abs(predict_data['PredictedPlayoffNextYear'] - predict_data['PlayOffNextYear'])
total_error = predict_data['Error'].sum()
print("Errore totale:", total_error)


Accuracy: 0.46153846153846156
Precision: 0.5555555555555556
Recall: 0.625
F1 Score: 0.5882352941176471

Classification Report:
               precision    recall  f1-score   support

         0.0       0.25      0.20      0.22         5
         1.0       0.56      0.62      0.59         8

    accuracy                           0.46        13
   macro avg       0.40      0.41      0.41        13
weighted avg       0.44      0.46      0.45        13

Squadre previste per i playoff nell'anno 10:
    tmID  confID  PlayoffProbability  AdjustedPlayoffProbability  \
126  SEA       1                0.80                    0.839895   
62   LAS       1                0.75                    0.787402   
116  SAS       1                0.55                    0.577428   
106  SAC       1                0.52                    0.545932   
52   IND       0                0.74                    0.776903   
11   CHI       0                0.64                    0.671916   
84   NYL       0        

<ipython-input-19-bf7b08d3d1ab>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_data['PlayoffProbability'] = model.predict_proba(X_predict)[:, 1]
<ipython-input-19-bf7b08d3d1ab>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_data['AdjustedPlayoffProbability'] = 8 * predict_data['PlayoffProbability'] / predict_data['PlayoffProbability'].sum()
<ipython-input-19-bf7b08d3d1ab>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [ ]:
# TODO list per la avalicao
# commentare correttamente tutto il codice
# provare a considerare in maniera diversa i premi di ogni player all'interno del team - per influenzare eventuali future vittorie
# provare più metodi di classification !!!
# aggiustare win rate
# iniziare bozza presentazione
# scegliere main topics presentazione
# mettere nella presentazione la cosa delle scommesse
# mettere in ordine i folders a sinistra
# shift della colonna dei playoff - vedi foto
# eseguzione pipeline - run file per lanciare tutto
# tante belle cose